In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv("bank.csv",sep=";")

In [2]:
train_cols = df.columns[0:-1]
label = df.columns[-1]
X = df[train_cols]
y = df[label].apply(lambda x: 0 if x == "no" else 1) # 0為沒有簽定期存款(term deposit), 1為有存款 
X_encodings = pd.get_dummies(X, prefix_sep='.')
feature_names = list(X_encodings.columns)

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X_encodings, y, test_size=0.20, random_state=seed)


# 訓練一般可解釋regression模型(LinearRegression、RegressionTree)

In [3]:
from interpret import show
from interpret.perf import RegressionPerf
from interpret.glassbox import LinearRegression, RegressionTree



lr = LinearRegression(random_state=seed)
lr.fit(X_train, y_train)

rt = RegressionTree(random_state=seed)
rt.fit(X_train, y_train)


lr_perf = RegressionPerf(lr.predict).explain_perf(X_test, y_test, name='Linear Regression')
rt_perf = RegressionPerf(rt.predict).explain_perf(X_test, y_test, name='Regression Tree')


In [4]:
lr_global = lr.explain_global(name='Linear Regression Global')
rt_global = rt.explain_global(name='Regression Tree Global')

lr_local = lr.explain_local(X_test[:10], y_test[:10], name='Linear Regression Local')
rt_local = rt.explain_local(X_test[:10], y_test[:10],name='Regression Tree local')

In [5]:
lr_perf = RegressionPerf(lr.predict).explain_perf(X_test, y_test, name='Linear Regression performance')
rt_perf = RegressionPerf(rt.predict).explain_perf(X_test, y_test, name='Regression Tree performance')

# Train an EBM Model

In [6]:
from interpret.glassbox import ExplainableBoostingRegressor
ebm = ExplainableBoostingRegressor(random_state=seed)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays


ebm_global = ebm.explain_global(name='EBM global')
ebm_local = ebm.explain_local(X_test[:5], y_test[:5], name='EBM local')
ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_test, y_test, name='EBM perf')

# 顯示全部

In [7]:
show([lr_perf,rt_perf,lr_global,rt_global,lr_local,rt_local,ebm_global ,ebm_local,ebm_perf])

<!-- http://127.0.0.1:7802/1645294906760/ -->
 Open in new window

# 顯示Morris Sensitivty for 3 models

In [8]:
from interpret.blackbox import MorrisSensitivity

sensitivity_lr = MorrisSensitivity(predict_fn=lr.predict, data=X_train)
sensitivity_lr_global = sensitivity_lr.explain_global(name="Global Sensitivity for linear r")

sensitivity_rt = MorrisSensitivity(predict_fn=rt.predict, data=X_train)
sensitivity_rt_global = sensitivity_rt.explain_global(name="Global Sensitivity for regression tree")

sensitivity_ebm = MorrisSensitivity(predict_fn=ebm.predict, data=X_train)
sensitivity_ebm_global = sensitivity_ebm.explain_global(name="Global Sensitivity for EBM")

show([sensitivity_lr_global,sensitivity_rt_global,sensitivity_ebm_global])

<!-- http://127.0.0.1:7802/1645350613640/ -->
 Open in new window